In [7]:
# Week 4 Individual Assignment
# Mapping Land use and zoning in San Francisco (February 2020) - SF Zoning Geojson
# the data is current as of February 2020, which means we can assume the land uses were current as of the 5th RHNA cycle starting in 2014
# make land use map interactive to isolate different land uses - esp industrial
# trying to over lay the different zones onto census tracts for comparison 
# to asthma rates and historic redline maps
# On July 18, 2013, the ABAG (Association for Bay Area Governments) issued the final RHNA allocations to jurisdictions with the deadline of having their updated housing elements approved by HCD by January 31, 2015. SF's 2014 Housing Element was approved on May 29, 2015.
# As such, I've used the 2015 zoning map to compare against asthma and health insurance data from the 2010 census. 

In [8]:
#importing libraries including Plotly to make this an interactive map
import pandas as pd
import geopandas as gpd
import plotly.express as px

In [9]:
#loading the 2015 Zoning Maps' CSV data file as pandas to insepct the data
df = pd.read_csv('data/SFZoningDistricts_Jan2015.csv')

In [10]:
#inspecting the shape of the csv file
df.shape

(9978, 7)

In [11]:
df.head

<bound method NDFrame.head of                                          districtna    shape_area  \
0           NEIGHBORHOOD COMMERCIAL, MODERATE SCALE  13901.142691   
1           NEIGHBORHOOD COMMERCIAL, MODERATE SCALE  11413.265361   
2           NEIGHBORHOOD COMMERCIAL, MODERATE SCALE  14482.491966   
3           NEIGHBORHOOD COMMERCIAL, MODERATE SCALE  54869.300002   
4           NEIGHBORHOOD COMMERCIAL, MODERATE SCALE  12643.761802   
...                                             ...           ...   
9973                    EXCELSIOR OUTER MISSION NCD  15828.010444   
9974                    EXCELSIOR OUTER MISSION NCD  24179.203550   
9975                    EXCELSIOR OUTER MISSION NCD  31868.200193   
9976                    EXCELSIOR OUTER MISSION NCD  23104.315653   
9977  JUDAH STREET NEIGHBORHOOD COMMERCIAL DISTRICT  24098.930935   

        shape_len                                                url  \
0      552.715582  http://www.amlegal.com/nxt/gateway.dll/Califor... 

In [12]:
# Looking at the data types
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9978 entries, 0 to 9977
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   districtna  9978 non-null   object 
 1   shape_area  9978 non-null   float64
 2   shape_len   9978 non-null   float64
 3   url         9909 non-null   object 
 4   zoning_sim  9978 non-null   object 
 5   geometry    9978 non-null   object 
 6   multigeom   9978 non-null   bool   
dtypes: bool(1), float64(2), object(4)
memory usage: 477.6+ KB


In [13]:
# Defining the columns to keep. 
# I'm only interested in the zoning district name, zoning use abbreviation, and geometry.
columns_to_keep = ['districtna',
                   'zoning_sim',
                   'geometry']
    

In [14]:
# Adding these columns to a new dataframe
df2 = df[columns_to_keep]

In [17]:
df2.head()

districtna zoning_sim  \
0  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE       NC-3   
1  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE       NC-3   
2  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE       NC-3   
3  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE       NC-3   
4  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE       NC-3   

                                            geometry  
0  POLYGON ((-122.39202280469173 37.7301014042066...  
1  POLYGON ((-122.39165098276553 37.7308690893996...  
2  POLYGON ((-122.39133947496012 37.7316878251099...  
3  POLYGON ((-122.39187443781078 37.7319472980624...  
4  POLYGON ((-122.39102797517454 37.7326246507840...

In [18]:
# Renaming these columns by creating a list first.
columns = list(df2)

In [19]:
columns

['districtna', 'zoning_sim', 'geometry']

In [22]:
# Renaming the columns. 
df2.columns = ['Zoning District Name',
               'Zoning Code',
               'Geometry']

In [23]:
df2.head()

Zoning District Name Zoning Code  \
0  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE        NC-3   
1  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE        NC-3   
2  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE        NC-3   
3  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE        NC-3   
4  NEIGHBORHOOD COMMERCIAL, MODERATE SCALE        NC-3   

                                            Geometry  
0  POLYGON ((-122.39202280469173 37.7301014042066...  
1  POLYGON ((-122.39165098276553 37.7308690893996...  
2  POLYGON ((-122.39133947496012 37.7316878251099...  
3  POLYGON ((-122.39187443781078 37.7319472980624...  
4  POLYGON ((-122.39102797517454 37.7326246507840...

In [36]:
# I'm most interested in Industrial (M) and Production,Pistribution, and Repair (PDR) Districts.
# So I'm running a query to see how many of these districts exist in San Francisco.

df2[df2['Zoning Code']=='M-1'] #M-1 Light Industrial

Zoning District Name Zoning Code  \
384      LIGHT INDUSTRIAL         M-1   
385      LIGHT INDUSTRIAL         M-1   
386      LIGHT INDUSTRIAL         M-1   
387      LIGHT INDUSTRIAL         M-1   
443      LIGHT INDUSTRIAL         M-1   
...                   ...         ...   
8578     LIGHT INDUSTRIAL         M-1   
8579     LIGHT INDUSTRIAL         M-1   
8580     LIGHT INDUSTRIAL         M-1   
8581     LIGHT INDUSTRIAL         M-1   
9886     LIGHT INDUSTRIAL         M-1   

                                               Geometry  
384   POLYGON ((-122.39275843595581 37.7522126952264...  
385   POLYGON ((-122.39619779392089 37.7833229405218...  
386   POLYGON ((-122.3898386804618 37.78622400234997...  
387   POLYGON ((-122.38833232203362 37.7888568704103...  
443   POLYGON ((-122.39426078453877 37.7437264114775...  
...                                                 ...  
8578  POLYGON ((-122.38844341686486 37.7400056049772...  
8579  POLYGON ((-122.38927684646023 37.7393650816683...  
8580  POLYGON ((-122.38912403033775 37.7398007233391...  
8581  POLYGON ((-122.3888236837157 37.74065692673501...  
9886  POLYGON ((-122.40136236168539 37.7820226114884...  

[70 rows x 3 columns]

In [39]:
# There are 70 different Light Industrial zones.
# Settting Light Indutrial as a new variable.
df2_M1 = df2[df2['Zoning Code']=='M-1']

In [40]:
df2_M1.count()

Zoning District Name    70
Zoning Code             70
Geometry                70
dtype: int64

In [41]:
 # I'm now running the same query but for Heavy Industrial Zones (M-2)
df2[df2['Zoning Code']=='M-2'] #M-2 Heavy Industrial

Zoning District Name Zoning Code  \
409      HEAVY INDUSTRIAL         M-2   
449      HEAVY INDUSTRIAL         M-2   
461      HEAVY INDUSTRIAL         M-2   
462      HEAVY INDUSTRIAL         M-2   
463      HEAVY INDUSTRIAL         M-2   
464      HEAVY INDUSTRIAL         M-2   
529      HEAVY INDUSTRIAL         M-2   
1744     HEAVY INDUSTRIAL         M-2   
1827     HEAVY INDUSTRIAL         M-2   
5300     HEAVY INDUSTRIAL         M-2   
5401     HEAVY INDUSTRIAL         M-2   
5591     HEAVY INDUSTRIAL         M-2   
6188     HEAVY INDUSTRIAL         M-2   
6190     HEAVY INDUSTRIAL         M-2   
7524     HEAVY INDUSTRIAL         M-2   
7526     HEAVY INDUSTRIAL         M-2   
7530     HEAVY INDUSTRIAL         M-2   
7531     HEAVY INDUSTRIAL         M-2   
7532     HEAVY INDUSTRIAL         M-2   
7540     HEAVY INDUSTRIAL         M-2   
7541     HEAVY INDUSTRIAL         M-2   
7563     HEAVY INDUSTRIAL         M-2   
7564     HEAVY INDUSTRIAL         M-2   
7575     HEAVY INDUSTRIAL         M-2   
7576     HEAVY INDUSTRIAL         M-2   
7577     HEAVY INDUSTRIAL         M-2   
7640     HEAVY INDUSTRIAL         M-2   
7678     HEAVY INDUSTRIAL         M-2   
7680     HEAVY INDUSTRIAL         M-2   
7712     HEAVY INDUSTRIAL         M-2   
7713     HEAVY INDUSTRIAL         M-2   
7714     HEAVY INDUSTRIAL         M-2   
7715     HEAVY INDUSTRIAL         M-2   
7716     HEAVY INDUSTRIAL         M-2   
7761     HEAVY INDUSTRIAL         M-2   
7763     HEAVY INDUSTRIAL         M-2   
7807     HEAVY INDUSTRIAL         M-2   
7815     HEAVY INDUSTRIAL         M-2   
7816     HEAVY INDUSTRIAL         M-2   
7817     HEAVY INDUSTRIAL         M-2   
7818     HEAVY INDUSTRIAL         M-2   
7925     HEAVY INDUSTRIAL         M-2   
7926     HEAVY INDUSTRIAL         M-2   
7952     HEAVY INDUSTRIAL         M-2   
7962     HEAVY INDUSTRIAL         M-2   
7963     HEAVY INDUSTRIAL         M-2   
7967     HEAVY INDUSTRIAL         M-2   
7971     HEAVY INDUSTRIAL         M-2   
8001     HEAVY INDUSTRIAL         M-2   
8028     HEAVY INDUSTRIAL         M-2   
8029     HEAVY INDUSTRIAL         M-2   
8030     HEAVY INDUSTRIAL         M-2   
8142     HEAVY INDUSTRIAL         M-2   
8232     HEAVY INDUSTRIAL         M-2   
8234     HEAVY INDUSTRIAL         M-2   
8299     HEAVY INDUSTRIAL         M-2   
8536     HEAVY INDUSTRIAL         M-2   

                                               Geometry  
409   POLYGON ((-122.37484255344206 37.7259917734637...  
449   POLYGON ((-122.38705252390402 37.7095866435619...  
461   POLYGON ((-122.3890805373271 37.70918320751531...  
462   POLYGON ((-122.38787681839399 37.7088522346776...  
463   POLYGON ((-122.40122620578438 37.7094683201652...  
464   POLYGON ((-122.3903306189682 37.70930226341549...  
529   POLYGON ((-122.38547971922087 37.7101064686518...  
1744  POLYGON ((-122.3882398818063 37.74886232398955...  
1827  POLYGON ((-122.38648809715662 37.7479633957502...  
5300  POLYGON ((-122.38393304759767 37.7518558494613...  
5401  POLYGON ((-122.38296708961904 37.7519115688148...  
5591  POLYGON ((-122.38203501710785 37.7429814317060...  
6188  POLYGON ((-122.38711582354134 37.7501794291538...  
6190  POLYGON ((-122.39174304882148 37.7521105384993...  
7524  POLYGON ((-122.38284172140932 37.7506503502964...  
7526  POLYGON ((-122.39306428779679 37.7500341337428...  
7530  POLYGON ((-122.39154281353115 37.7471415836458...  
7531  POLYGON ((-122.39160877253357 37.7472300661169...  
7532  POLYGON ((-122.39231248775562 37.7472259325644...  
7540  POLYGON ((-122.370111693164 37.728823554946715...  
7541  POLYGON ((-122.36973698021893 37.7325585681899...  
7563  POLYGON ((-122.3720236986755 37.73421223199656...  
7564  POLYGON ((-122.3749500817155 37.73697783518743...  
7575  POLYGON ((-122.38489900417325 37.7518001239926...  
7576  POLYGON ((-122.3810351681361 37.7520229819872,...  
7577  POLYGON ((-122.38649321499267 37.7581400689617...  
7640  POLYGON ((-122.39350090479793 37.7498023910218...  
76

In [42]:
df2_M2 = df2[df2['Zoning Code']=='M-2'] 

In [43]:
df2_M2.count()

Zoning District Name    57
Zoning Code             57
Geometry                57
dtype: int64

In [45]:
import geopandas as gpd

In [ ]:
industry=gpd.read_file